In [3]:
import requests
import pandas as pd

def get_references(doi):
    """
    Retrieves outgoing references for the given DOI.
    For a DOI (e.g., doi_1) that cites other works, this endpoint returns records
    where the input DOI is the citing paper and the returned 'cited' field is the target.
    """
    url = f"https://opencitations.net/index/api/v2/references/doi:{doi}"
    response = requests.get(url)
    edges = []
    if response.status_code == 200:
        data = response.json()
        # Each record in the response should contain (at least) a field 'cited'
        # which is the DOI of the referenced (cited) work.
        for record in data:
            # For outgoing references, the origin is our input DOI.
            cited_doi = record.get("cited")
            if cited_doi:
                edges.append({
                    "origin_doi": doi,
                    "target_doi": cited_doi
                })
    else:
        print(f"Error {response.status_code} when fetching references for {doi}")
    return edges

def get_citations(doi):
    """
    Retrieves incoming citations for the given DOI.
    For a DOI (e.g., doi_1) that is cited by other papers, this endpoint returns records
    where the returned 'citing' field is the DOI of the citing work.
    """
    url = f"https://opencitations.net/index/api/v2/citations/doi:{doi}"
    response = requests.get(url)
    edges = []
    if response.status_code == 200:
        data = response.json()
        # Each record in the response should include a 'citing' field.
        for record in data:
            citing_doi = record.get("citing")
            if citing_doi:
                edges.append({
                    "origin_doi": citing_doi,  # the work that cites our DOI
                    "target_doi": doi           # our input DOI is being cited
                })
    else:
        print(f"Error {response.status_code} when fetching citations for {doi}")
    return edges


In [4]:
# Example DOI – replace with your actual DOI string.
doi_input = "10.1016/j.neucom.2018.10.063"

# Get outgoing and incoming edges.
outgoing_edges = get_references(doi_input)
incoming_edges = get_citations(doi_input)

# Combine both lists.
all_edges = outgoing_edges + incoming_edges

# Build the DataFrame and drop any duplicate rows.
df_citations = pd.DataFrame(all_edges).drop_duplicates()

Citation edges:
                                           origin_doi  \
0                        10.1016/j.neucom.2018.10.063   
1                        10.1016/j.neucom.2018.10.063   
2                        10.1016/j.neucom.2018.10.063   
3                        10.1016/j.neucom.2018.10.063   
4                        10.1016/j.neucom.2018.10.063   
5                        10.1016/j.neucom.2018.10.063   
6                        10.1016/j.neucom.2018.10.063   
7                        10.1016/j.neucom.2018.10.063   
8                        10.1016/j.neucom.2018.10.063   
9                        10.1016/j.neucom.2018.10.063   
10                       10.1016/j.neucom.2018.10.063   
11                       10.1016/j.neucom.2018.10.063   
12                       10.1016/j.neucom.2018.10.063   
13                       10.1016/j.neucom.2018.10.063   
14                       10.1016/j.neucom.2018.10.063   
15  omid:br/06102484665 doi:10.1007/978-981-13-909...   
16  omid:br/061

In [6]:
df_citations

,origin_doi,target_doi
0,10.1016/j.neucom.2018.10.063,omid:br/061101954495 doi:10.1109/tsmcb.2011.21...
1,10.1016/j.neucom.2018.10.063,omid:br/06110204893 doi:10.1016/j.neunet.2014....
2,10.1016/j.neucom.2018.10.063,omid:br/061302451105 doi:10.1109/ijcnn.2017.79...
3,10.1016/j.neucom.2018.10.063,omid:br/061502085732 doi:10.1155/2016/3049632 ...
4,10.1016/j.neucom.2018.10.063,omid:br/062202408220 doi:10.1364/josaa.2.00116...
5,10.1016/j.neucom.2018.10.063,omid:br/062301812569 doi:10.1007/s11045-016-04...
6,10.1016/j.neucom.2018.10.063,omid:br/062401755848 doi:10.1109/tnnls.2014.23...
7,10.1016/j.neucom.2018.10.063,omid:br/062503104995 doi:10.1109/tip.2015.2475...
8,10.1016/j.neucom.2018.10.063,omid:br/062602376972 doi:10.1109/robomech.2017...
9,10.1016/j.neucom.2018.10.063,omid:br/0627021793 doi:10.1016/j.neucom.2006.0...
